<a href="https://colab.research.google.com/github/vamsiSLU/HDS-5210/blob/main/midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [17]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    (string, int, float,float, float, float, float, string, string, string) -> float

    The function calculates the PRIEST COVID-19 Clinical Severity Score to assess a patient's risk of adverse outcomes.
    This function combines clinical and patient demographic data to provide a score that is then converted to a risk probability.
    If an incorrect value for the patient's sex is entered, the function throws a ValueError. The scoring is based on the criteria from the MDCalc website.
    After computing the score based on the patient's clinical parameters, the function converts that score into a probability of adverse outcomes within 30 days.
    The final result is a single probability value that reflects the patient's risk, derived using specific scoring thresholds.

    Args:
        sex (str): Gender assigned at birth,  'male' or 'female'.
        age (int): Age in years.
        respiratory_rate (float): Respiratory rate in breaths per minute.
        oxygen_saturation (float): Oxygen saturation which is a decimal between 0 and 1.
        heart_rate (float): Heart rate measured as beats per minute.
        systolic_bp (float): Systolic blood pressure in mmHg.
        temperature (float): Body temperature of patient in degrees Celsius.
        alertness (str): Status of alertness,  'alert' or 'confused or not alert'.
        inspired_oxygen (str): oxygen inspired,  'air' or 'supplemental oxygen'.
        performance_status (str): Unrestricted normal activity, Limited strenuous activity, can do light activity	Limited activity, can self-care	Limited self-care	Bed/chair bound, no self-care

    Returns:
        float: The 30-day probability of an adverse clinical outcome.

    Examples:

        >>> priest('male', 55, 22, 0.93, 100, 100, 37.5, 'alert', 'air', 'limited strenuous activity, can do light activity')
        0.34

        >>> priest('male', 45, 8, 0.90, 108, 200, 37.9, 'confused or not alert', 'air', 'limited activity, can self-care')
        0.46

        >>> priest('female', 85, 30, 0.88, 50, 85, 34.0, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
        0.99
    """
    # Converting string inputs to lowercase for case-insensitive comparisons.
    sex = sex.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()

    # Validating the inputs
    if sex not in ['male', 'female']:
        raise ValueError("Invalid sex input. Must be 'male' or 'female'.")

    # Start calculating the PRIEST score
    priest_score = 0

    # Sex
    if sex == 'male':
        priest_score += 1

    # Age
    if age > 80:
        priest_score += 4
    elif age > 65:
        priest_score += 3
    elif age > 50:
        priest_score += 2

    # Respiratory rate
    if respiratory_rate > 24:
        priest_score += 3
    elif respiratory_rate > 20:
        priest_score += 2
    elif respiratory_rate < 9:
        priest_score += 3
    elif respiratory_rate <= 11:
        priest_score += 1

    # Oxygen saturation
    if oxygen_saturation < 0.92:
        priest_score += 3
    elif oxygen_saturation <= 0.93:
        priest_score += 2
    elif oxygen_saturation <= 0.95:
        priest_score += 1

    # Heart rate
    if heart_rate > 130:
        priest_score += 3
    elif heart_rate > 110:
        priest_score += 2
    elif heart_rate > 90:
        priest_score += 1
    elif heart_rate < 41:
        priest_score += 3
    elif heart_rate <= 50:
        priest_score += 1

    # Systolic blood pressure
    if systolic_bp > 219:
        priest_score += 3
    elif systolic_bp < 91:
        priest_score += 3
    elif systolic_bp <= 100:
        priest_score += 2
    elif systolic_bp <= 110:
        priest_score += 1

    # Body Temperature of the patient
    if temperature > 39.0:
        priest_score += 2
    elif temperature >= 38.1:
        priest_score += 1
    elif temperature < 35.1:
        priest_score += 3
    elif temperature <= 36.0:
        priest_score += 1

    # Alertness
    if alertness != 'alert':
        priest_score += 3

    # Inspired oxygen
    if inspired_oxygen == 'supplemental oxygen':
        priest_score += 2

    # Performance status
    if performance_status == 'bed/chair bound, no self-care':
        priest_score += 4
    elif performance_status == 'limited self-care':
        priest_score += 3
    elif performance_status == 'limited activity, can self-care':
        priest_score += 2
    elif performance_status == 'limited strenuous activity, can do light activity':
        priest_score += 1

    # Converting the PRIEST score into probability
    if priest_score >= 26:
        return 0.99
    elif priest_score >= 17:
        return 0.59
    elif priest_score == 16:
        return 0.55
    elif priest_score == 15:
        return 0.49
    elif priest_score == 14:
        return 0.47
    elif priest_score == 13:
        return 0.46
    elif priest_score == 12:
        return 0.38
    elif priest_score == 11:
        return 0.34
    elif priest_score == 10:
        return 0.29
    elif priest_score == 9:
        return 0.26
    elif priest_score == 8:
        return 0.22
    elif priest_score == 7:
        return 0.18
    elif priest_score == 6:
        return 0.15
    elif priest_score == 5:
        return 0.09
    elif priest_score == 4:
        return 0.03
    elif 2 <= priest_score <= 3:
        return 0.02
    elif 0 <= priest_score <= 1:
        return 0.01
    else:
        return "Score is not calculated"



In [18]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('male', 55, 22, 0.93, 100, 100, 37.5, 'alert', 'air', 'limited strenuous activity, can do light activity')
Expecting:
    0.34
ok
Trying:
    priest('male', 45, 8, 0.90, 108, 200, 37.9, 'confused or not alert', 'air', 'limited activity, can self-care')
Expecting:
    0.46
ok
Trying:
    priest('female', 85, 30, 0.88, 50, 85, 34.0, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')  
Expecting:
    0.99
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [19]:
import requests

def find_hospital(age, sex, risk):
    """
    (int, string, int) -> string

   Using the REST web service to identify a suitable hospital based on the patient's age, sex, and risk parameters.
   Returns the hospital name or None if there is an error or no hospital is found.


    Args:
        age (int): The age of the patient.
        sex (str): The sex of the patient ('male' or 'female'). Case insensitive.
        risk (float): The risk percentage as a decimal.

    Returns:
        str: The name of the recommended hospital or None if no hospital is found or an error occurs.

    Examples:
        >>> find_hospital(25, 'male', 0.56)
        'Emory Dunwoody Medical Center'

        >>> find_hospital(45, 'male', 0.29)
        'Southwest Hospital and Medical Center'

        >>> find_hospital(52, 'male', 0.47)
        'Emory Dunwoody Medical Center'
    """


    # Adjust the sex input to ensure it aligns with the format required by the web service.
    sex = sex.lower()
    if sex not in ('male', 'female'):
        # Return None if the sex input is invalid
        return None


    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}"

    try:
        # Make the GET request
        results = requests.get(url)
        results.raise_for_status()  # This will raise HTTPError for bad responses

        # Process the JSON responsefor retrieving the hospital name
        response_data = results.json()
        hospital_name = response_data.get("hospital")

        return hospital_name if hospital_name else None

    except requests.RequestException as e:

        print(f"Request error: {e}")
        return None
    except ValueError as e:
        # Manage the errors related to JSON decoding
        print(f"JSON decoding error: {e}")
        return None



In [20]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(25, 'male', 0.56)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(45, 'male', 0.29)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(52, 'male', 0.47)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [21]:
import requests

def get_address(hospital_name):
    """
    (string) -> string

    Get the address of a hospital from JSON resource using the hospital's name.
    Managing errors like HTTP issues, JSON decoding problems, and missing data, returning None if any errors arise.
    The hospital name is trimmed and converted to uppercase to ensure it matches the dataset regardless of case sensitivity.



    Args:
        hospital_name (str): The name of the hospital
    Returns:
        str or None: Returns the hospital address, else, returns None.

    Examples:
        >>> get_address("Emory Dunwoody Medical Center")
        '4500 NORTH SHALLOWFORD ROAD'

        >>> get_address("Southwest Hospital and Medical Center")
        '501 FAIRBURN ROAD SW'

        >>> get_address("PIEDMONT HOSPITAL")
        '1968 PEACHTREE ROAD NW'


    """


    json_url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

    try:
        # Fetch the data from the URL
        results = requests.get(json_url)
        results.raise_for_status()  # Raises an exception for bad responses
        # Analyzing the JSON response
        hospital_data = results.json()

        # Normalizing the hospital name by stripping whitespace and converting it to uppercase.
        normalize_name = hospital_name.strip().upper()

        # Return the hospital address if found
        return hospital_data.get(normalize_name, {}).get("ADDRESS")
    except requests.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except ValueError as e:
        print(f"Error decoding JSON: {e}")
    except KeyError as e:
        print(f"Data formatting error: {e}")
# Return none if hospital address is not found
    return None


In [22]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("Emory Dunwoody Medical Center")
Expecting:
    '4500 NORTH SHALLOWFORD ROAD'
ok
Trying:
    get_address("Southwest Hospital and Medical Center")
Expecting:
    '501 FAIRBURN ROAD SW'
ok
Trying:
    get_address("PIEDMONT HOSPITAL")
Expecting:
    '1968 PEACHTREE ROAD NW'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [23]:
import requests
import csv

def process_people(file_url):
    """
    Processing patient data from the URL that contains a PSV (pipe-separated values) file.
    Assessing clinical risks, determining the hospitals, and retrieves their addresses.

    Args:
        file_url (str): URL to the PSV file that has the patient data.

    Returns:
        dict: A dictionary that associates patient identifiers with a list of patient information,
        which includes calculated risk, hospital name, and address.

    Resulting dictionary format:
    {patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}
    """
    try:
        results = requests.get(file_url)
        results.raise_for_status()
        print("Started processing the patients")

        # Split the response text into lines and parse the PSV data
        lines = results.text.strip().splitlines()
        data = csv.reader(lines, delimiter='|')

        header = next(data)

        patient_results = {}
        for row in data:
            if len(row) == 11:
                patient_number, sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status = row

                # Calculate risk
                risk = priest(sex, int(age), int(breath), float(o2sat), int(heart),
                              int(systolic), float(temp), alertness, inspired, status)

                # Find hospital and address
                hospital_name = find_hospital(int(age), sex.lower(), risk)
                hospital_address = get_address(hospital_name)

                patient_results[patient_number] = [sex, int(age), float(breath), float(o2sat), int(heart), int(systolic),
                                           float(temp), alertness, inspired, status, risk, hospital_name, hospital_address]

        print("Finished processing the patients")
        return patient_results

    except requests.RequestException as e:
        print(f"Failed to retrieve data: {e}")
        return {}
    except Exception as e:
        print(f"Error occurred during processing: {e}")
        return {}


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [24]:
import json
import requests

# URL of the file having expected results
url = 'https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5'

results = requests.get(url)
if results.status_code == 200:
    expected_results = json.loads(results.text)
else:
    print(f"Failed to download data from {url}")
    expected_results = {}

# URL of the file having the results
results_data = process_people("https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh")

# Verifying whether the hospital name and address correspond to the expected results.
if expected_results:
    for patient_number, result_values in results_data.items():
        if patient_number in expected_results:
            expected_values = expected_results[patient_number]
            if len(expected_values) > 11 and len(result_values) > 11:
                hospital_match = (expected_values[11] == result_values[11])
                if len(expected_values) > 12 and len(result_values) > 12:
                    address_match = (expected_values[12] == result_values[12])

                    if hospital_match and address_match:
                        print(f"Patient {patient_number} - All results match the expected outcomes.")
                    elif not hospital_match:
                        print(f"Patient {patient_number} - The hospital name does not match the expected value.")
                    elif not address_match:
                        print(f"Patient {patient_number} - The address of the hospital does not match the expected value.")
                else:
                    print(f"Patient {patient_number} - Not enough data to compare the address.")
            else:
                print(f"Patient {patient_number} - Not enough data to compare hospital name.")
        else:
            print(f"Patient {patient_number} is not listed in the expected results.")
else:
    print("No reliable expected results are available for comparison.")


Started processing the patients
Finished processing the patients
Patient E9559 - All results match the expected outcomes.
Patient E9385 - All results match the expected outcomes.
Patient E3067 - All results match the expected outcomes.
Patient E9422 - All results match the expected outcomes.
Patient E8661 - All results match the expected outcomes.
Patient E6235 - All results match the expected outcomes.
Patient E4451 - All results match the expected outcomes.
Patient E8433 - All results match the expected outcomes.
Patient E7593 - All results match the expected outcomes.
Patient E3296 - All results match the expected outcomes.
Patient E4157 - All results match the expected outcomes.
Patient E7702 - All results match the expected outcomes.
Patient E8158 - All results match the expected outcomes.
Patient E4795 - All results match the expected outcomes.
Patient E8800 - All results match the expected outcomes.
Patient E4855 - All results match the expected outcomes.
Patient E9691 - All res

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---